In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.notebook import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma/keras/gemma_2b_en/2/config.json
/kaggle/input/gemma/keras/gemma_2b_en/2/tokenizer.json
/kaggle/input/gemma/keras/gemma_2b_en/2/metadata.json
/kaggle/input/gemma/keras/gemma_2b_en/2/model.weights.h5
/kaggle/input/gemma/keras/gemma_2b_en/2/assets/tokenizer/vocabulary.spm
/kaggle/input/llm-prompt-recovery/sample_submission.csv
/kaggle/input/llm-prompt-recovery/train.csv
/kaggle/input/llm-prompt-recovery/test.csv
/kaggle/input/databricks-dolly-15k/README.md
/kaggle/input/databricks-dolly-15k/databricks-dolly-15k.jsonl
/kaggle/input/gemma-rewrite-nbroad/nbroad-v2.csv
/kaggle/input/gemma-rewrite-nbroad/nbroad-v1.csv


In [2]:
# !pip install -U keras-nlp
# !pip install -U keras

In [3]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [4]:
import keras
import keras_nlp

2024-04-02 11:36:52.020503: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 11:36:52.020657: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 11:36:52.166753: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [6]:
data = pd.read_csv("/kaggle/input/gemma-rewrite-nbroad/nbroad-v2.csv")
data.drop(["id"], axis=1)

,original_text,rewrite_prompt,rewritten_text
0,"`` Well, there are healthier ways to tell me y...",Rewrite the story where the writer asks the re...,"Well, there are healthier ways to tell me you ..."
1,Rory ran his shaky fingers through his wife's ...,Rewrite the essay as a dramatic play,## The Final Curtain\n\n[FADE IN]\n\n**Setting...
2,As I made my way on foot across town to the Po...,Rewrite the story with all the themes and sett...,As I made my way through the Tatooine desert o...
3,`` Hello. We come in peace.'' \n \n The first ...,Rewrite the essay if the advanced aliens didn'...,`` Hello. We come in peace.''\n\nThe first enc...
4,"`` Karen, what the helllllll izzz...'' says my...",Rewrite the story as a court room drama starri...,The courtroom erupted in an uproar as District...
...,...,...,...
2395,"`` Dave, have you seen this?'' There is no ans...",Rewrite this as a Shakespearean tragedy,"Alas, poor Dave, a victim of fate's cruel hand..."
2396,"I posted something like this a while ago, I gu...",Rewrite the story as if you are a famous celeb...,"I'm the epitome of stardom, yet I can't shake ..."
2397,"`` Oh Darling, it was absolutely dreadful!'' M...",Rewrite the story as an upbeat mystery story (...,"Hey folks, buckle up for a mystery that will h..."
2398,Ubwuwuaua! I exclaim shaking my head back and ...,Rewrite the story from the perspective of the ...,Ubwuwuaua. The whispers of the wind dance thro...


In [7]:
def gen_data(original_text, rewritten_text):
    text = f"""    
    Original Essay:
    \"""{original_text}\"""
    
    Rewritten Essay:
    \"""{rewritten_text}\"""
    
    Given are 2 essays, the Rewritten essay was created from the Original essay using the google Gemma model.
    You are trying to understand how the original essay was transformed into a new version.
    Analyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten essay.
    Start directly with the prompt, that's all I need. Output should be only line ONLY.
    """
    return text

data['pred'] = list(map(gen_data, data['original_text'], data['rewritten_text']))
data.head()

,id,original_text,rewrite_prompt,rewritten_text,pred
0,lZGdiueMer,"`` Well, there are healthier ways to tell me y...",Rewrite the story where the writer asks the re...,"Well, there are healthier ways to tell me you ...","\n Original Essay:\n """"""`` Well, the..."
1,DfTJVFKrUk,Rory ran his shaky fingers through his wife's ...,Rewrite the essay as a dramatic play,## The Final Curtain\n\n[FADE IN]\n\n**Setting...,"\n Original Essay:\n """"""Rory ran his..."
2,LmJvKranXK,As I made my way on foot across town to the Po...,Rewrite the story with all the themes and sett...,As I made my way through the Tatooine desert o...,"\n Original Essay:\n """"""As I made my..."
3,PpnqXQAdGH,`` Hello. We come in peace.'' \n \n The first ...,Rewrite the essay if the advanced aliens didn'...,`` Hello. We come in peace.''\n\nThe first enc...,"\n Original Essay:\n """"""`` Hello. We..."
4,qOeXTfqgAM,"`` Karen, what the helllllll izzz...'' says my...",Rewrite the story as a court room drama starri...,The courtroom erupted in an uproar as District...,"\n Original Essay:\n """"""`` Karen, wh..."


In [8]:
gemma_lm.backbone.enable_lora(rank=32)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,517,084,160 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,517,084,160 (9.38 GB)

 Trainable params: 10,911,744 (41.62 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [9]:
train_data = data['pred'].tolist()
train_data[:1]

['    \n    Original Essay:\n    """`` Well, there are healthier ways to tell me you do n\'t like it, but I guess this is what I get for soliciting feedback. \n \n `` Jimmy, there are two things I really need you to do. The first is that I need you to get the hell out of my house, go home, and fuck yourself with one of the many empty wine bottles you keep in your apartment. Do n\'t bother lubing it up, lube is for closers. \n \n `` The second thing I need you to do, after cleaning up the blood of course, is to hop on Google and look up Dropbox. What did you think this is, the Seventies? You\'d pull some\'Misery\' bullshit and torch my book because it did n\'t have enough pictures for you? I\'ve got three revision passes worth of text on my hard drive, a fourth on a USB drive, and all that and a bag of chips on the Cloud. And since it turns out that your knuckles drag even lower than I originally thought, I do n\'t mean the things in the sky that look like rabbits, you troglodyte. \n \n

In [10]:
#due to the limitation of training time, thereby only training 1000 row of data
train_data = train_data[:1000]
len(train_data)

1000

In [11]:
gemma_lm.preprocessor.sequence_length = 512

optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(train_data, epochs=1, batch_size=1)

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 755s 733ms/step - loss: 2.5559 - sparse_categorical_accuracy: 0.4802


In [12]:
test_df = pd.read_csv("/kaggle/input/llm-prompt-recovery/test.csv")
test_df['original_text'] = test_df['original_text'].fillna("")
test_df['rewritten_text'] = test_df['rewritten_text'].fillna("")
test_df.head()

,id,original_text,rewritten_text
0,-1,The competition dataset comprises text passage...,Here is your shanty: (Verse 1) The text is rew...


In [13]:
row = test_df.iloc[0]

template = """Instruction:\nBelow, the `Original Text` passage has been rewritten/transformed/improved into `Rewritten Text` by the `Gemma 2b` LLM with a certain prompt/instruction. Your task is to carefully analyze the differences between the `Original Text` and `Rewritten Text`, and try to infer the specific prompt or instruction that was likely given to the LLM to rewrite/transform/improve the text in this way.\n\nOriginal Text:\n{original_text}\n\nRewriten Text:\n{rewritten_text}\n\nResponse:\n{rewrite_prompt}"""
prompt = template.format(
    original_text=row.original_text,
    rewritten_text=row.rewritten_text,
    rewrite_prompt="",
)
output = gemma_lm.generate(prompt, max_length=512)


In [14]:
print(output)

Instruction:
Below, the `Original Text` passage has been rewritten/transformed/improved into `Rewritten Text` by the `Gemma 2b` LLM with a certain prompt/instruction. Your task is to carefully analyze the differences between the `Original Text` and `Rewritten Text`, and try to infer the specific prompt or instruction that was likely given to the LLM to rewrite/transform/improve the text in this way.

Original Text:
The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text.  Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set.

Rewriten Text:
Here is your shanty: (Verse 1) The text is rewritten, the LLM has spun, With prompts so clever, they've been outrun. The goal is to find, 

In [15]:
preds = []
for i in tqdm(range(len(test_df))):
    row = test_df.iloc[i]

    # Generate Prompt using template
    prompt = template.format(
        original_text=row.original_text,
        rewritten_text=row.rewritten_text,
        rewrite_prompt=""
    )

    output = gemma_lm.generate(prompt, max_length=512)
    pred = output.replace(prompt, "") # remove the prompt from output
    
    preds.append([row.id, pred])

  0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
sub_df = pd.DataFrame(preds, columns=["id", "rewrite_prompt"])
sub_df['rewrite_prompt'] = sub_df['rewrite_prompt'].fillna("")
sub_df['rewrite_prompt'] = sub_df['rewrite_prompt'].map(lambda x: "Improve the essay" if len(x) == 0 else x)
sub_df.to_csv("submission.csv",index=False)
sub_df.head()

,id,rewrite_prompt
0,-1,Improve the essay
